In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import logging.config
import os
import pickle
from copy import deepcopy

import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import wildfires.utils as utils
from wildfires.analysis.plotting import (
    cube_plotting,
    map_model_output,
    partial_dependence_plot,
)
from wildfires.analysis.processing import log_map, log_modulus, map_name, vif
from wildfires.data.cube_aggregation import *
from wildfires.data.datasets import *
from wildfires.data.datasets import DATA_DIR, data_map_plot
from wildfires.logging_config import LOGGING

logger = logging.getLogger(__name__)
logging.config.dictConfig(LOGGING)

In [ ]:
normal_size = 9.0
normal_coast_linewidth = 0.5
dpi = 600

In [ ]:
selection = Datasets() + ERA5_DryDayPeriod() + GSMaP_dry_day_period()
selection.show("pretty")

min_time, max_time, times_df = dataset_times(selection.datasets)
print(times_df.to_string(index=False))
# print(times_df.to_latex(index=False))

monthly_datasets, mean_datasets, climatology_datasets = prepare_selection(selection)

In [ ]:
# Get land mask.
land_mask = ~utils.land_mask(n_lon=1440)

# Define a latitude mask which ignores data beyond 60 degrees, as the precipitation data does not extend to those latitudes.
lats = mean_datasets.cubes[0].coord("latitude").points
lons = mean_datasets.cubes[0].coord("longitude").points

latitude_grid = np.meshgrid(lats, lons, indexing="ij")[0]
lat_mask = np.abs(latitude_grid) > 60

# Make a deep copy so that the original cubes are preserved.
lat_land_datasets = mean_datasets.copy(deep=True)

for cube in lat_land_datasets.cubes:
    cube.data.mask[lat_mask] = True
    cube.data.mask[land_mask] = True

mpl.rcParams["figure.figsize"] = (11, 6)

for dataset in lat_land_datasets:
    # cube_plotting(dataset.cubes[0],log=True, title=f"log Mean Dry Day Period ({dataset.name})")
    cube_plotting(
        dataset.cubes[0], log=False, title=f"Mean Dry Day Period ({dataset.name})"
    )